# Giai đoạn 1: Chuẩn bị data

Bước 1: Import data

In [ ]:
import pandas as pd

books = pd.read_csv("/content/drive/MyDrive/Dataset/archive/Books.csv")
ratings = pd.read_csv("/content/drive/MyDrive/Dataset/archive/Ratings.csv")
users = pd.read_csv("/content/drive/MyDrive/Dataset/archive/Users.csv")

books.head()

In [ ]:
ratings.head()

In [ ]:
users.head()

Bước 2: Tạo df là bảng Rating kết trái với bảng Book

In [ ]:
df = ratings.merge(books, how="left", on="ISBN")

df.head()

Bước 3: Đếm số người rating đối với từng quyển sách. Lưu tên những quyển sách được ít hơn 15 người vote để xóa khỏi dataset

In [ ]:
rating_count = pd.DataFrame(df["Book-Title"].value_counts())

rating_count.head()

rare_rating_count chứa những quyển sách được ít hơn 600 user rate

In [ ]:
rare_rating_count = rating_count[rating_count["Book-Title"] < 600].index
rare_rating_count.size

common_rating_count là bảng kết trái ở trên nhưng lọc bỏ những dòng có chứa những quyển ít được user đánh giá ở trên

In [ ]:
common_rating_count = df[~df["Book-Title"].isin(rare_rating_count)]
common_rating_count

Bước 4: Tạo một pivot table (book_df), chỉ số dòng là user-id, chỉ số cột là tên sách. Mỗi giá trị trên một cell đại diện cho số rating của một user cho một quyển sách

In [ ]:
book_df = common_rating_count.pivot_table(index=["User-ID"], columns=["Book-Title"], values="Book-Rating")
book_df.head()

# Giai đoạn 2: Xác định các bộ phim được xem bởi user

Bước 1: Chọn random một user

In [ ]:
random_user = 230522

Bước 2: Tạo một dataframe (random_user_df) từ pivot table chứa những rating đối với những quyển sách mà user đó đã xem

In [ ]:
random_user_df = book_df[book_df.index == random_user]
random_user_df

Bước 3: Gán những quyển mà đã được coi (hoặc rating) bởi user đó vào list books_read

In [ ]:
books_read = random_user_df.columns[random_user_df.notna().any()].tolist()
books_read

# Giai đoạn 3: Trích xuất ra các similar user

Bước 1: Lọc bỏ những quyển mà random user chưa đọc khỏi pivot table bằng cách chỉ giữ lại những cột có trong books_read, gán dataframe này cho books_read_df

In [ ]:
books_read_df = book_df[books_read]
books_read_df.head()

Bước 2: Tạo một dataframe (user_book_count) để tính xem với mỗi user (những user còn lại ngoài random user) đọc được bao nhiêu quyển trong những quyển mà thằng random user đã đọc

In [ ]:
user_book_count = books_read_df.T.notnull().sum()
user_book_count = user_book_count.reset_index()
user_book_count.columns = ["User-ID", "Book-Count"]
user_book_count

Bước 3: Giả sử những user mà đọc được ít nhất 60% tổng số cuốn mà random user đã đọc thì coi những user này là những thằng user giống với random_user. Giờ ta sẽ tạo series users_same_books để chứa id những user này

In [ ]:
perc = len(books_read) * 55 / 100
users_same_books = user_book_count[user_book_count["Book-Count"] >= perc]["User-ID"]
users_same_books

# Giai đoạn 4: Tìm ra những user giống với random user nhất

Bước 1: Lọc từ dataframe books_read_df những similar user đã tìm được ở trên rồi gán vào final_df

In [ ]:
final_df = pd.concat([books_read_df[books_read_df.index.isin(users_same_books)], random_user_df[books_read]])
final_df

Bước 2: Tạo bảng corr_df chứa mối quan hệ giữa những similar user với random user với nhau

In [ ]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ["user_id_1", "user_id_2"]
corr_df = corr_df.reset_index()

corr_df

Bước 3: Tạo bảng top_users chứa những user có mối quan hệ > 0.55 so với random user

In [ ]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] > 0.55)][["user_id_2", "corr"]].reset_index(drop=True)
top_users

Bước 4: Merge bảng top_users với bảng ratings -> bảng top_users_ratings. Ý nghĩa cái bảng này là chỉ ra những thằng top_users rating những quyển sách như thế nào

In [ ]:
top_users = top_users.sort_values(by="corr", ascending=False)
top_users.rename(columns={"user_id_2": "User-ID"}, inplace=True)
final = top_users.merge(ratings[["User-ID", "ISBN", "Book-Rating"]], how="inner")
top_users_ratings = final[final["User-ID"] != random_user]
top_users_ratings

# Giai đoạn 5: Dự đoán random_user sẽ rating những quyển sách giá trị bao nhiêu và lấy top 5 quyển có giá trị rating cao nhất

Bước 1: Tạo thêm một cột ở bảng trên (top_users_ratings) cột "weighted_rating" được tính bằng corr*Book-Rating. Thông số này có ý nghĩa là dự đoán random_user sẽ rating những quyển sách với con số tương ứng

In [ ]:
top_users_ratings["weighted_rating"] = top_users_ratings["corr"] * top_users_ratings["Book-Rating"]
top_users_ratings

Bước 2: Tạo dataframe recommendation_df là tương ứng với mỗi quyển sách thì weighted_rating là nhiêu, ý là random user rating books giá trị weighted_rating

In [ ]:
recommendation_df = top_users_ratings.groupby("ISBN").agg({"weighted_rating": "mean"})
recommendation_df

Bước 3: Lọc ra những quyển được rating trên 6

In [ ]:
books_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 6].sort_values("weighted_rating", ascending=False)
books_to_be_recommend = books_to_be_recommend.reset_index()
books_to_be_recommend

Bước 4: Gắn thông tin sách lên id của sách ở bảng trên vào bảng books_to_be_recommended_info

In [ ]:
books_to_be_recommend_info = books_to_be_recommend.merge(books[["ISBN", "Book-Title"]])
books_to_be_recommend_info

Bước 5: Lấy top 5 quyển là xong

In [ ]:
recommended = list(books_to_be_recommend_info["Book-Title"].loc[0:5])
recommended